# CM4AI U2OS Croissant Builder 🥐

Author: Ido Diamant, Ma'ayan Lab, CFDE DRC

In [ ]:
import datetime
import hashlib
import json
import mlcroissant as mlc
import os
import pandas as pd
import requests
from zipfile import ZipFile

## Construct Croissant from CFDE Attribute Table

In [ ]:
name='CM4AI U2OS Cell Map Protein Localization Assemblies'
description="Protein localization assemblies constructed from integrating AP-MS biomolecular interaction and IF imaging data"

In [ ]:
short_citation = 'Schaffer, Nature, 2025'
title = 'Multimodal cell maps as a foundation for structural and functional genomics'
author = 'Schafffer, LV'
journal = 'Nature'
year = 2025
volume = 642
pages = '222–231'

cite_as=(f'@article{{{short_citation}, title={{{title}}}, author={{{author}}}, journal={{{journal}}}, year={{{year}}}, volume={{{volume}}}, pages={{{pages}}}}}')

In [ ]:
creators = [
    mlc.Organization(name="Cell Maps for AI", url="https://cm4ai.org/")
]

publishers = [
    mlc.Organization(name="Ma'ayan Lab", url="https://maayanlab.cloud/")
]

license = "http://creativecommons.org/licenses/by-nc-sa/4.0"
version = "0.1.0"
url="https://maayanlab.cloud/Harmonizome/dataset/CM4AI+U2OS+Cell+Map+Protein+Localization+Assemblies"
date_published=datetime.date(2025, 9, 29)

### Resource

In [ ]:
# Utility function to generate a SHA256 checksum for a FileObject from a URL
def get_sha256(url):
    sha256 = hashlib.sha256()
    response = requests.get(url, stream=True)
    for chunk in response.iter_content(chunk_size=65536):
        sha256.update(chunk)
    return sha256.hexdigest()

In [ ]:
file_url = 'https://maayanlab.cloud/static/hdfs/harmonizome/data/cm4aiu2os/gene_attribute_matrix.txt.zip'

distribution = [
    mlc.FileObject(
        id="dataset-attribute-table-archive",
        name="dataset-attribute-table-archive",
        description="Dataset attribute table archive from Harmonizome.",
        content_url=file_url,
        encoding_formats="application/zip",
        sha256=get_sha256(file_url)
    ),
    mlc.FileObject(
        id="dataset-attribute-table",
        name="dataset-attribute-table",
        description="Dataset attribute table from Harmonizome.",
        content_url="gene_attribute_matrix.txt",
        encoding_formats="text/tab-separated-values",
        contained_in={"dataset-attribute-table-archive"}
    )
]

### Structure/Semantics

In [ ]:
matrix = pd.read_csv(file_url, sep='\t', compression='zip')
display(matrix)

,Gene,9-1-1-RAD17-RFC complex,AP-1 transcription factor complex,Actin cytoskeleton remodeling and signal transduction 1,Actin cytoskeleton remodeling and signal transduction 2,Actin filaments,Aminoacyl-tRNA synthetase multienzyme assembly,Amyloid precursor protein complex 1,Amyloid precursor protein complex 2,Anaphase-promoting and cell cycle kinase complexes,...,mRNA Surveillance and Repair Pathway,mRNA cleavage and polyadenylation specificity factor complex,mRNA regulation and decay,mRNA regulation complex,mRNA surveillance and decay pathway,proton-transporting V-type ATPase complex,snRNP assembly,tRNA protein synthesis complex,tRNA splicing endonuclease and telomere cap assembly,tRNA-splicing ligase complex
0,AAAS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AAGAB,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AAK1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AAMDC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AAMP,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4830,ZW10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4831,ZWINT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4832,ZYG11B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4833,ZYX,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
fields = []
array_size = matrix.shape[0]

fields.append(
  mlc.Field(
    id="associations/gene",
    name="gene",
    description="The NCBI gene symbol",
    data_types=mlc.DataType.TEXT,
    is_array=True,
    array_shape=str(array_size),
    source=mlc.Source(
      file_object="dataset-attribute-table",
      extract=mlc.Extract(column="Gene")
    )
  )
)

for col in matrix.columns[1:]:
  fields.append(
    mlc.Field(
      id=f"associations/{col.replace(' ','_')}",
      name=f"associations/{col}",
      data_types=mlc.DataType.INTEGER,
      is_array=True,
      array_shape=str(array_size),
      source=mlc.Source(
        file_object="dataset-attribute-table",
        extract=mlc.Extract(column=col)
      )
    )
  )

print(len(fields))

272


In [ ]:
record_sets = [
    mlc.RecordSet(
        id="associations",
        name="associations",
        key='associations/gene',
        fields = fields
    )
]

### Assemble Dataset and Write JSON
Once we have defined each dataset layer, we can package the dataset metadata into a Croissant JSON-LD file.

In [ ]:
metadata = mlc.Metadata(
    name = name,
    description = description,
    cite_as = cite_as,
    url = url,
    date_published=date_published,
    creators = creators,
    publisher = publishers,
    license = license,
    version = version,
    distribution = distribution,
    record_sets = record_sets
)

# Display any warnings/suggestions encountered by the validator when building the metadata
print(metadata.issues.report())

In [ ]:
with open("cm4aiu2os.json", "w") as f:
  content = metadata.to_json()
  content = json.dumps(content, indent=4, default=str)
  f.write(content)
  f.write("\n")  # Terminate file with newline